# 应用遗传算法寻找八数码从初始状态到目标状态的解路径

> 3220191000 软件工程 赵菊文

In [18]:
# 导入算法需要的库以及定义相关变量
import  math
import  random
import  copy

C = 540                           # 最大适应度
LEN = 140                         # 基因长度
maxGene = []                      # 最长基因组
maxi = 0                          # 最大值最初出现的进化代数
findAim = False                   # 是否找到目标（用于评价）
POPMAX = 32                       # 种群数量
P_XOVER = 0.8                     # 交叉概率
P_MUTATION = 0.15                 # 变异概率
MAXGENERATIONS = 1000             # 总的进化代数
goal = [1,2,3,8,0,4,7,6,5]        # 目标状态
initiate = [2,8,3,1,0,4,7,6,5]    # 初始状态
pop = []                          # 种群所有对象

## 1. 个体编码

遗传算法的运算对象是表示个体的符号串，所以必须把变量 x1, x2 编码为一种符号串。本题中，用无符号二进制整数来表示。因 x1, x2 为 0 ~ 7之间的整数，所以分别用3位无符号二进制整数来表示，将它们连接在一起所组成的6位无符号二进制数就形成了个体的基因型，表示一个可行解。
例如，基因型 X＝101110 所对应的表现型是：x＝[ 5，6 ]。个体的表现型x和基因型X之间可通过编码和解码程序相互转换。

In [17]:
class Gene:
  def __init__(self, gene):
      self.gene = gene  
      self.fitness = 0
      self.rf = 0    
      self.cf = 0    

## 2. 初始群体产生

遗传算法是对群体进行的进化操作，需要给其淮备一些表示起始搜索点的初始群体数据。本例中，群体规模的大小取为4，即群体由4个个体组成，每个个体可通过随机方法产生。如：011101，101011，011100，111001。

In [7]:
def initGenes() :
  count = 0
  maxFit = 100                                # 随机生成的基因适应度的最大值
  while(count < POPMAX):
      tmp = []
      for j in range(LEN):
          pow = round(random.random() * 3)    # 随机生成算法：0 -> 上，1 -> 下, 2 -> 左, 3 -> 右
          tmp.append(pow)
 
      pop.append(Gene(tmp))
      count+=1
        

def move (current, dire):
     space = 0 
     block = 0 
     for i in range(len(current)):
        if (current[i] == 0):
            space = i
            block = space
            break
 
 
     if (dire == 0) :
        if (space - 3 >= 0):
            block = space - 3
 
     elif(dire == 1 and (space + 3 < 9)) :
            block  = space + 3
     elif(dire == 2) :
          if (space % 3 > 0) :
             block = space - 1
 
     elif(dire == 3) :
        if (space % 3 < 2) :
            block = space + 1
 
 
     current[space], current[block]= current[block], current[space]
     if (space == block):
        return False
     else:
        return True

## 3. 适应度计算

遗传算法中以个体适应度的大小来评定各个个体的优劣程度，从而决定其遗传机会的大小。本例中，目标函数总取非负值，并且是以求函数最大值为优化目标，故可直接利用目标函数值作为个体的适应度。

In [8]:
def fitness(current) :
    f = 0
    for i in range(len(current)):
        if (current[i] == goal[i]) :
            f += 100 - current[i]*10
    return f

## 4. 选择运算

选择运算(或称为复制运算)把当前群体中适应度较高的个体按某种规则或模型遗传到下一代群体中。一般要求适应度较高的个体将有更多的机会遗传到下一代群体中。本例中，我们采用与适应度成正比的概率来确定各个个体复制到下一代群体中的数量。操作过程的具体描述如下：

1. 先计算出群体中所有个体的适应度的总和；
2. 其次计算出每个个体的相对适应度的大小，它即为每个个体被遗传到下一代群体中的概率；
3. 每个概率值组成一个区域，全部概率值之和为1；
4. 最后再产生一个0到1之间的随机数，依据该随机数出现在上述哪一个概率区域内来确定各个个体被选中的次数。

In [19]:
def envaluateFitness(maxi):          # max 参数用于记录进化代数
  totalFitness = 0
  for i in range(POPMAX):
      s0 = initiate[:]               # 每一步移动后的状态
      pop[i].fitness = 0
      for j in range(LEN):
        move(s0, pop[i].gene[j])
        pop[i].fitness = fitness(s0)
        if (pop[i].fitness == C) :
            global findAim
            findAim = True
            global maxGene
            maxGene = pop[i].gene[0:j+1]
            return totalFitness
 
 
      if(pop[i].fitness == 0) :
        pop[i].fitness = 1
 
      totalFitness += pop[i].fitness
 
  return totalFitness

def selectBetter(totalFitness):                  # 轮盘赌选择算法简单实现
  lastCf = 0
  newPop = [None for i in range(POPMAX)]
  global pop
  for i in range(POPMAX):                        # 计算个体选择概率和累积概率
      pop[i].rf = pop[i].fitness / totalFitness
      pop[i].cf = lastCf + pop[i].rf
      lastCf = pop[i].cf
 
  for i in range(POPMAX):                        #轮盘赌式选择
      p = random.random()
      if(p < pop[0].cf):
          newPop[i] = Gene(pop[0].gene)
          # newPop.append(Gene(pop[0].gene))
      else:
          for j in range(POPMAX-1):
              if(p >= pop[j].cf and p < pop[j+1].cf):
                  newPop[i] = Gene(pop[j+1].gene)
                  # newPop.append(Gene(pop[j+1].gene))
                  break
 
  if (len(newPop) == 0) :
      # console.log(pop)
      print(pop)
  pop = copy.deepcopy(newPop)

def exChgOver(first,second):         
  ecc = round(random.random() * LEN)
  for i in range(int(ecc)):
      index = math.floor(random.random() * LEN)
      pop[first].gene[index], pop[second].gene[index] = pop[second].gene[index], pop[first].gene[index]

## 5. 交叉运算

交叉运算是遗传算法中产生新个体的主要操作过程，它以某一概率相互交换某两个个体之间的部分染色体。本例采用单点交叉的方法，其具体操作过程是：

1. 先对群体进行随机配对；
2. 其次随机设置交叉点位置；
3. 最后再相互交换配对染色体之间的部分基因。

In [20]:
def crossover():
  first = -1
  for i in range(POPMAX):
      p = random.random()
      if(p < P_XOVER):
          if(first < 0):
              first = i
          else:   
              exChgOver(first,i)
              first = -1

## 6. 变异运算

变异运算是对个体的某一个或某一些基因座上的基因值按某一较小的概率进行改变，它也是产生新个体的一种操作方法。本例中，我们采用基本位变异的方法来进行变异运算，其具体操作过程是：

1. 首先确定出各个个体的基因变异位置，下表所示为随机产生的变异点位置，其中的数字表示变异点设置在该基因座处；
2. 然后依照某一概率将变异点的原有基因值取反。

In [21]:
def reverseGene(index):         # 变异操作函数
  mcc = round(random.random() * LEN)
  for i in range(int(mcc)):
      gi = math.floor(random.random() * LEN)
      pop[index].gene[gi] = 3 - pop[index].gene[gi]
 
 
def mutation():
  for i in range(POPMAX):
      p = random.random()
      if(p < P_MUTATION):       # 只有当随机数小于变异概率才进行变异操作
          reverseGene(i)

## 7. 算法示例输出

In [22]:
def transform (gene) :
    s0 = initiate[:]
    options = []
    for i in range(len(gene)):
        if (move(s0, gene[i])) :
            if (gene[i] == 0) :
                options.append('上')
            elif (gene[i] == 1) :
                options.append('下')
            elif (gene[i] == 2) :
                options.append('左')
            elif (gene[i] == 3):
                options.append('右')
 
    print(options)
    print(s0)
    
initGenes()
f = envaluateFitness(0)
for i in range(MAXGENERATIONS):
    selectBetter(f)
    crossover()
    mutation()
    f= envaluateFitness(i)
    if (findAim) :
        break
        
transform(maxGene)

['右', '左', '下', '左', '右', '左', '上', '右', '左', '下', '上', '下', '上', '下', '上', '右', '下', '上', '左', '上', '下', '右', '左', '右', '左', '下', '上', '上', '下', '下', '右', '右', '左', '上', '下', '左', '右', '上', '上', '右', '下', '左', '右', '上', '左', '右', '左', '左', '下', '下', '上', '右']
[1, 2, 3, 8, 0, 4, 7, 6, 5]
